for breif matrix explanations read-> https://github.com/USEPA/USEEIO_API/blob/master/doc/data_format.md

In [2]:
#BASE URL AND KEY
#Use the requests library
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
#Set the base URL.
#base_url = 'https://api.edap-cluster.com/useeio/api'
base_url ='https://smmtool.app.cloud.gov/api/' 
#Add a key for the request hhttps://api.edap-cluster.com/useeio/api/eaders. No keys are needed by default, so it's blank
api_headers = {}
# api-key for USEEIO
# api_headers['x-api-key']='cjdNxlaMdY4B0C84v2Nua3O0rubhHj421K0Z318l'
# api-key for USEEIO
with open(".USEEIO_API_KEY", 'r') as KEY_FILE:
    api_headers['x-api-key']=KEY_FILE.read()

In [3]:
#AVAILABLE MODELS
#Show the available models in the native JSON output
models = r.get(base_url+'/models',headers=api_headers)
models_json = models.json()
models_json
#Returns output like
#[{'id': 'USEEIOv1.1',
#  'name': 'USEEIOv1.1',
#  'location': 'US',
#  'description': 'EPA national life cycle model of goods and services, v1.1.'}]

[{'id': 'USEEIO', 'name': 'USEEIO', 'location': 'US'},
 {'id': 'GAUSEEIO', 'name': 'GAUSEEIO', 'location': 'GA'}]

##### Select here of you want to work with GA or the whole USA data

In [4]:
#MODEL NAME
#Use the first available model
model_name = models_json[1]['id']
print('The first model name is ' + str(model_name) + '.')

The first model name is GAUSEEIO.


In [5]:
#INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
HC_indictr_id = model_indictrs_json[8]['id']
model_indictrs_names = []
model_indictrs_ids = []
for i in model_indictrs_json:
    model_indictrs_names.append(i['name'])
    model_indictrs_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_indictrs_json)) + ' indicators. The names are:')
print(model_indictrs_names)
HTOX_indictr_id = model_indictrs_json[11]['id']
#print(HTOX_indictr_id)

The GAUSEEIO model has 24 indicators. The names are:
['Acid Rain', 'Energy', 'Freshwater Aquatic Ecotoxicity', 'Eutrophication', 'Food Waste', 'Global Climate Change', 'Hazardous Air Pollutants', 'Hazardous Waste', 'Human Health Cancer', 'Human Health Noncancer', 'Human Health - Respiratory Effects', 'Human Health Cancer and Noncancer', 'Employment', 'Land', 'Metals', 'Minerals and Metals', 'Muncipal Solid Waste', 'Nonrenewable Energy', 'Ozone Depletion', 'Pesticides', 'Renewable Energy', 'Smog Formation', 'Value Added', 'Water']


In [7]:
dqname = '(Reliability,Time Corr,Geo Corr,Tech Corr,Data Collection)'
#MODEL SECTORS
#Get the sectors
model_sectors = r.get(base_url+model_name+'/sectors',headers=api_headers)
model_sectors_json = model_sectors.json()
model_sector_ids = []
model_sectors_json
for i in model_sectors_json:
    model_sector_ids.append(i['id'])
first_sector_id = model_sector_ids[0]
first_sector_name = model_sectors_json[0]['name']
print('The '+ model_name +' model has ' + str(len(model_sectors_json)) + ' sectors.')
print('The first sector name is \"' + first_sector_name + '\" with an ID of \"' + first_sector_id + '\"')

The GAUSEEIO model has 776 sectors.
The first sector name is "Fresh soybeans, canola, flaxseeds, and other oilseeds" with an ID of "1111a0/fresh soybeans, canola, flaxseeds, and other oilseeds/us-ga"


In [30]:
[_ for _ in model_sector_ids if (_[:4] == "1111") & (_[-4:] != "rous")]

['1111a0/fresh soybeans, canola, flaxseeds, and other oilseeds/us-ga',
 '1111b0/fresh wheat, corn, rice, and other grains/us-ga']

In [31]:
# model_sector_ids

## Top sectors related to Human Cancer and HTOX (Per dollar value)

In [34]:
#DIRECT IMPACTS
#Get top direct impacts for HC(Human Cancer) indicator
#Query the first row of the direct impacts matrix, D. This provides the direct impacts for the first indicator.
D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
D0 = D0_response.json()
#Get the accompanying data quality scores
D0dqi_response = r.get(base_url+model_name+'/matrix/D_dqi?row=0',headers=api_headers)
try:
    D0dqi = D0dqi_response.json()
except: 
    D0dqi = [0]*len(D0)
#Convert it to a dataframe
D0_df = pd.DataFrame({HC_indictr_id:D0},index=model_sector_ids)
#Sort it by descending order
# D0_df = D0_df.sort_values(by=HC_indictr_id,ascending=False)
print('Table. Top 10 direct sector impacts per dollar for ' + model_indictrs_names[8] +' with DQ scores.')
# D0_df.head(10)

# D0_df.loc[]

Table. Top 10 direct sector impacts per dollar for Human Health Cancer with DQ scores.


In [36]:
D0_df.loc[

#     [_ for _ in model_sector_ids if (_[:4] == "1111") & (_[-4:] != "rous")]

    [_ for _ in model_sector_ids if (_[-4:] != "rous")]

]

,HC
"1111a0/fresh soybeans, canola, flaxseeds, and other oilseeds/us-ga",0.006991
"1111b0/fresh wheat, corn, rice, and other grains/us-ga",0.023198
"111200/fresh vegetables, melons, and potatoes/us-ga",0.005637
111300/fresh fruits and tree nuts/us-ga,0.003150
"111400/greenhouse crops, mushrooms, nurseries, and flowers/us-ga",0.000780
...,...
s00402/used and secondhand goods/us-ga,0.000000
s00500/federal general government (defense)/us-ga,0.000019
s00600/federal general government (nondefense)/us-ga,0.000005
s00700/state and local general government/us-ga,0.000025


In [52]:
#DIRECT IMPACTS
#Get top direct impacts for HTOX indicator
#Query the first row of the direct impacts matrix, D. This provides the direct impacts for the first indicator.
D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
D0 = D0_response.json()
#Get the accompanying data quality scores
D0dqi_response = r.get(base_url+model_name+'/matrix/D_dqi?row=0',headers=api_headers)
try:
    D0dqi = D0dqi_response.json()
except: 
    D0dqi = [0]*len(D0)
#Convert it to a dataframe
D0_df = pd.DataFrame({HTOX_indictr_id:D0},index=model_sector_ids)
#Sort it by descending order
D0_df = D0_df.sort_values(by=HTOX_indictr_id,ascending=False)
print('Table. Top 10 direct sector impacts per dollar for ' + model_indictrs_names[11] +' with DQ scores.')
D0_df.head(10)

Table. Top 10 direct sector impacts per dollar for Human Health Cancer and Noncancer with DQ scores.


,HTOX
112a00/animal farms and aquaculture ponds (except cattle and poultry)/rous,0.030414
327310/cement/rous,0.024351
112a00/animal farms and aquaculture ponds (except cattle and poultry)/us-ga,0.023216
"1111b0/fresh wheat, corn, rice, and other grains/us-ga",0.023198
"483000/water transport (boats, ships, ferries)/rous",0.022553
221100/electricity/us-ga,0.022001
112300/poultry farms/rous,0.021694
327310/cement/us-ga,0.021472
112300/poultry farms/us-ga,0.020613
"1111b0/fresh wheat, corn, rice, and other grains/rous",0.019810


## Top sectors related to Human Cancer  Per demand vector
#### What even is demand vector?!
#### Why is this only 2007?!

In [53]:
#DEMAND VECTORS
#See the demand vectors available for the model in their JSON format
demands_response = r.get(base_url+model_name+'/demands',headers=api_headers)
demands = demands_response.json()
demands

[{'id': '2007_ga_consumption',
  'year': 2007,
  'type': 'Consumption',
  'system': 'Full System',
  'location': 'GA'},
 {'id': '2007_ga_production',
  'year': 2007,
  'type': 'Production',
  'system': 'Full System',
  'location': 'GA'},
 {'id': '2007_ga_consumption_food',
  'year': 2007,
  'type': 'Consumption',
  'system': 'Food System',
  'location': 'GA'},
 {'id': '2007_ga_production_food',
  'year': 2007,
  'type': 'Production',
  'system': 'Food System',
  'location': 'GA'},
 {'id': '2007_us_consumption',
  'year': 2007,
  'type': 'Consumption',
  'system': 'Full System',
  'location': 'US'},
 {'id': '2007_us_production',
  'year': 2007,
  'type': 'Production',
  'system': 'Full System',
  'location': 'US'},
 {'id': '2007_us_consumption_food',
  'year': 2007,
  'type': 'Consumption',
  'system': 'Food System',
  'location': 'US'},
 {'id': '2007_us_production_food',
  'year': 2007,
  'type': 'Production',
  'system': 'Food System',
  'location': 'US'}]

In [54]:
#DEMAND VECTOR IN NATIVE FORMAT
#See the first demand vector in its native format, which has to be created if one wants to calculate the model results with a custom vector
first_demand_id = demands[1]['id']
first_demand_id
y0_response = r.get(base_url+model_name+'/demands/'+first_demand_id,headers=api_headers)
y0 = y0_response.json()
if len(y0)>5:
    print(y0[0:5])
else:
    print(y0)

[{'sector': '1111a0/fresh soybeans, canola, flaxseeds, and other oilseeds/rous', 'amount': -6435810.84080927}, {'sector': '1111a0/fresh soybeans, canola, flaxseeds, and other oilseeds/us-ga', 'amount': -2829320.19067274}, {'sector': '1111b0/fresh wheat, corn, rice, and other grains/rous', 'amount': 195082127.60257}, {'sector': '1111b0/fresh wheat, corn, rice, and other grains/us-ga', 'amount': 62102321.6148822}, {'sector': '111200/fresh vegetables, melons, and potatoes/rous', 'amount': 324186911.141618}]


In [55]:
#SECTORS IN ROWS. 
#Use this same demand for a model calculation. Use the direct perspective calculation. 
#Show a transposed view with indicators in the columns and sectors in the rows
data_to_post = {"perspective": "direct"}
data_to_post["demand"] = y0
result1_response = r.post(base_url+model_name+'/calculate',headers=api_headers,json=data_to_post)
result1 = result1_response.json()
result1_df = pd.DataFrame(data=result1['data'],columns=result1['sectors'],index=result1['indicators']).transpose()
result1_df = result1_df.sort_values(by=HC_indictr_id,ascending=False)
print('Top ten sectors by direct perspective impact for indicator '+ model_indictrs_names[8] + ' using demand vector ' +  first_demand_id + ', with 4 additional indicator results presented.')
result1_df[model_indictrs_ids[8]].head(20)

Top ten sectors by direct perspective impact for indicator Human Health Cancer using demand vector 2007_ga_production, with 4 additional indicator results presented.


484000/truck transport/us-ga                                                                       6.727894
211000/unrefined oil and gas/rous                                                                  4.443556
484000/truck transport/rous                                                                        2.463153
233411/single-family residential structures/us-ga                                                  1.679057
486000/pipeline transport/us-ga                                                                    1.583266
111900/tobacco, cotton, sugarcane, peanuts, sugar beets, herbs and spices, and other crops/rous    1.286130
486000/pipeline transport/rous                                                                     0.997112
221100/electricity/us-ga                                                                           0.751990
48a000/scenic and sightseeing transportation and support activities for transportation/us-ga       0.704543
1111b0/fresh wheat, corn, ri

In [56]:
#SECTORS IN ROWS. 
#Use this same demand for a model calculation. Use the direct perspective calculation. 
#Show a transposed view with indicators in the columns and sectors in the rows
data_to_post = {"perspective": "direct"}
data_to_post["demand"] = y0
result1_response = r.post(base_url+model_name+'/calculate',headers=api_headers,json=data_to_post)
result1 = result1_response.json()
result1_df = pd.DataFrame(data=result1['data'],columns=result1['sectors'],index=result1['indicators']).transpose()
result1_df = result1_df.sort_values(by=HTOX_indictr_id,ascending=False)
print('Top ten sectors by direct perspective impact for indicator '+ model_indictrs_names[11] + ' using demand vector ' +  first_demand_id + ', with 4 additional indicator results presented.')
result1_df[model_indictrs_ids[11]].head(10)

Top ten sectors by direct perspective impact for indicator Human Health Cancer and Noncancer using demand vector 2007_ga_production, with 4 additional indicator results presented.


484000/truck transport/us-ga                                                                       33.170455
211000/unrefined oil and gas/rous                                                                  32.007642
48a000/scenic and sightseeing transportation and support activities for transportation/us-ga       20.745588
484000/truck transport/rous                                                                        12.233495
111900/tobacco, cotton, sugarcane, peanuts, sugar beets, herbs and spices, and other crops/rous     8.926833
486000/pipeline transport/rous                                                                      8.189080
1111b0/fresh wheat, corn, rice, and other grains/rous                                               5.656902
221100/electricity/us-ga                                                                            5.366512
322110/wood pulp/rous                                                                               5.179001
233411/single-famil